# Notebook 12: Link Spam and Topic-Specific PageRank
***

In this notebook we'll examine the how topic-specific PageRank can be leveraged to weed out spam pages and diagnose spam.

We'll need numpy for this notebook, so let's load it.

In [1]:
import numpy as np

<br>

### Exercise 0: Following along with lecture is cool!

With the random teleports (to solve the spider traps and dead ends problems), our transition matrix $A$ is constructed of:
$$A_{ij} = \beta M_{ij} + (1-\beta)/N$$
where $M$ is the original transition matrix, $\beta$ is a probability (typically between 0.8 and 0.9) of taking an actual out-link (as opposed to a teleport), and $N$ is the number of pages. These elements $A_{ij}$ are the probability of transitioning from page $j$ to page $i$ (possibly a bit sideways from what you might intuitively think, but it makes the power iteration work out).

With ***topic-specific*** PageRank, however, we restrict the teleport set $S$ to only a subset of the pages (on the topic of interest). Our transition matrix becomes:
$$A_{ij} = \begin{cases} \beta M_{ij} + (1-\beta)/|S| & \text{if } i \in S \\ \beta M_{ij} & \text{otherwise} \end{cases}$$

For the example of slide 10, we have the following graph:

<img width=200px src="http://www.cs.colorado.edu/~anwo7157/home/resources/topicspecificpagerank1.png">

We saw that the transition matrix $M$ for this graph is:

In [2]:
M = np.array([[1/2, 1/3, 0, 1/2],
              [1/2, 1/3, 0, 0  ],
              [0  , 1/3, 0, 1/2],
              [0  , 0  , 1, 0  ]])

Using $\beta = 0.8$ and the teleport set $S = \{1,2\}$, we can make a slight modification of our previous code to make it topic-specific. In particular, in the matrix that is associated with the teleports (the $1-\beta$ term in $A$), we want 1s in the rows corresponding to the teleport set's pages, and 0s in the other rows. In our original code, we have 1s everywhere. We also need to account for division by the size of the teleport set, $|S|$, instead of the full set of pages.

In [3]:
# SOLUTION:

n = 4
beta = 0.8
Nmat = np.zeros((n,n))
S = [1, 2]
for i in S:
    Nmat[i-1,:] = np.ones(n)/len(S)
A = beta*M + (1-beta)*Nmat

To iterate, we need to initialize our PageRank vector, which we can do using an even distribution of rank:

In [4]:
r_old = np.repeat(1/n, n)

And we can perform 10 iterations of power iteration, and make sure we agree with the lecture results:

In [5]:
# SOLUTION:

print("    1      2      3      4")
print("{:2.0f}  {:0.3f}  {:0.3f}  {:0.3f}  {:0.3f}".format(0, r_old[0],r_old[1],r_old[2],r_old[3]))
for t in range(10):
    r_new = np.matmul(A, r_old)
    r_old = r_new.copy()
    # Renormalize r_new to make sure it sums to 1:
    #   1) obtain the sum of the elements in r_new
    mag = np.sum(r_new)
    #   2) divide each element of r_new by the sum of all of them
    r_new = r_new*(1/mag)
    print("{:2.0f}  {:0.3f}  {:0.3f}  {:0.3f}  {:0.3f}".format(t+1, r_new[0],r_new[1],r_new[2],r_new[3]))

    1      2      3      4
 0  0.250  0.250  0.250  0.250
 1  0.367  0.267  0.167  0.200
 2  0.398  0.318  0.151  0.133
 3  0.397  0.344  0.138  0.121
 4  0.399  0.351  0.140  0.110
 5  0.397  0.353  0.138  0.112
 6  0.398  0.353  0.139  0.110
 7  0.397  0.353  0.138  0.111
 8  0.398  0.353  0.139  0.111
 9  0.397  0.353  0.138  0.111
10  0.398  0.353  0.139  0.111


<br>

### Exercise 1: Spam farming

Recall the original graph we looked at when learning power iteration (notebook 10).

<img width=250px src="http://www.cs.colorado.edu/~anwo7157/home/resources/pagerank1.png">

Let's obtain the PageRanks for each of the 5 pages using a teleport probability of 0.15 ($\beta = 0.85$). We'll use the compact representation of the transition matrix $M$, since this will lend itself well to expanding our web graph as the devious owner of Page 4 starts up a **spam farm**!

In [6]:
# SOLUTION:

M_compact = [[1, [2]],
             [1, [5]],
             [2, [1, 4]],
             [3, [1, 3, 5]],
             [3, [1, 2, 3]]]

Stealing liberally from our own previous code, we can initialize and run 50 iterations of power iteration as follows:

In [7]:
# initialize for power iteration
n = len(M_compact)
beta = 0.85
r_old = np.repeat(1/n, n)

for t in range(50):
    # initialize with the "taxed" importance
    r_new = np.repeat((1-beta)/n, n)
    # distribute importance payments between the nodes
    for page in range(n):
        for dest in M_compact[page][1]:
            idx = dest-1 # accounting for Python's 0-based indexing
            r_new[idx] += beta*r_old[page]/M_compact[page][0]
    # normalize to ensure r_new sums to 1
    r_new = r_new/np.sum(r_new)
    r_old = r_new

print(np.round(r_new, 4))

[0.1974 0.2811 0.1385 0.0889 0.2941]


**Consider:** What line would we need to change if we wanted to use *topic-specific* PageRank? And why?

The devious owner of Page 4, Franklin, is sad that his page has such low rank. In an effort to boost it, he decides to create a **spam farm**. He doesn't want to go all in at once, so first Franklin creates a single spam page (Page 6), with a single out-link to Page 4, and a single in-link from Page 4.

Append the link information for Page 6 to `M_compact`. Don't forget to update the link information for Page 4, as well as Page 6.

In [8]:
# SOLUTION:

M_compact.append([1, [4]])
M_compact[3][1].append(len(M_compact))
M_compact[3][0] = len(M_compact[3][1])
print(M_compact)

[[1, [2]], [1, [5]], [2, [1, 4]], [4, [1, 3, 5, 6]], [3, [1, 2, 3]], [1, [4]]]


And recompute the PageRank for all of the pages. Don't forget to update the number of pages!

In [9]:
# initialize for power iteration
n = len(M_compact)
beta = 0.85
r_old = np.repeat(1/n, n)

for t in range(50):
    # initialize with the "taxed" importance
    r_new = np.repeat((1-beta)/n, n)
    # distribute importance payments between the nodes
    for page in range(n):
        for dest in M_compact[page][1]:
            idx = dest-1 # accounting for Python's 0-based indexing
            r_new[idx] += beta*r_old[page]/M_compact[page][0]
    # normalize to ensure r_new sums to 1
    r_new = r_new/np.sum(r_new)
    r_old = r_new

print(np.round(r_new, 4))

[0.1802 0.2537 0.1265 0.122  0.2666 0.0509]


We're going to need the full PageRank for all of these pages later, so let's hold onto it.

In [10]:
pagerank = r_new

By how much did the rank of Page 4 increase as a result of this spam farm page? The following equation was derived in class as the theoretical increase in PageRank of a target page as a result of $M$ spam pages feeding into it:

$$y = x + \beta M \left[\dfrac{\beta y}{M} + \dfrac{1-\beta}{N}\right] + \dfrac{1-\beta}{N}$$

where $y$ is the PageRank of the target page and $x$ is the rank contributed by good, upstanding pages. Is there some part of this equation that needs to be modified to fit our graph with the added spam page? Why or why not?

**Solution:** 

The term in square brackets represents the PageRank of each farm page. The first term in there, $\beta y/M$, is the rank contributed by the target page ***if the target page is only linking to spam farm pages***. So we need to update the $M$ in the denominator to $d_t$, the out-degree of the target page.

You should have found that the increase in PageRank for Franklin was large. Why did adding a single farm page yield such a huge benefit?

**Solution:**

There aren't very many pages total (only 5 legitimate ones), so adding a farm page to the mix might make a big difference! But right now, Franklin just shares probability with his farm page and doesn't get it back fast enough!  Combined, though, they're up to .17 compared to his prior total of .14.

<br>

### Exercise 2: Franklin's Treachery, and TrustRank, the hero we need!

<img width=280px src="http://www.cs.colorado.edu/~anwo7157/home/resources/linkspam1.png">

Now that our world is crashing down around us because of the treachery of Franklin, we need to figure out how to combat his admittedly sad little spam farm. Use the teleport set $S = \{1,2,3,4,5\}$ and $\beta = 0.85$ to perform 50 iterations of power iteration and estimate *TrustRank* with respect to the trusted set $S$ (the original pages). Note that you will likely need to change the "initialize with the taxed importance" step in the code we used previously, to account for teleports only to the trusted set.

In [11]:
import copy

In [31]:
# SOLUTION:

# initialize for power iteration
S = [1,2,3,4,5]
n = len(M_compact)
beta = 0.85
r_old = np.repeat(1/n, n)

for t in range(500):
    # initialize with the "taxed" importance
    r_new = np.array([(1-beta)/len(S) if p+1 in S else 0 for p in range(n)])
    # distribute importance payments between the nodes
    for page in range(n):
        for dest in M_compact[page][1]:
            idx = dest-1 # accounting for Python's 0-based indexing
            r_new[idx] += beta*r_old[page]/M_compact[page][0]
            #print(page, dest, r_old[idx],M_compact[page][0])
    # normalize to ensure r_new sums to 1
    r_new = r_new/np.sum(r_new)
    r_old = r_new

print(np.round(r_new, 4))

[0.1884 0.27   0.1322 0.1052 0.2819 0.0224]


In [24]:
print(np.array([(1-beta)/len(S) if p+1 in S else 0 for p in range(n)]))
print(M_compact)

[0.03 0.03 0.03 0.03 0.03 0.  ]
[[1, [2]], [1, [5]], [2, [1, 4]], [4, [1, 3, 5, 6]], [3, [1, 2, 3]], [1, [4]]]


The rank of Page 4 hasn't really decreased by very much, which might be discouraging. After all, if we haven't knocked Franklin down a peg, what is all this really about?

But wait! We can compute the **spam mass** of each page as:
$$m = \dfrac{r - t}{r}$$
where $r$ is the PageRank (including the spam farm page) and $t$ is the TrustRank (with respect to only the trusted set $S$).

So our spam masses for each page are:

In [32]:
spammass = (pagerank - r_new)/pagerank
print(spammass)

[-0.04537606 -0.06413099 -0.04537606  0.1381096  -0.05719823  0.56114813]


What do you notice about Franklin and his farm?